#### A simple Roberta hugging face NER based approach

Thanks for the notebook
https://www.kaggle.com/zzy990106/pytorch-ner-infer.

The training notebook can be found here
https://www.kaggle.com/revathiprakash/feedback-prize-baseline-ner-pytorch-train


In [ ]:
import random
import os
import time
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from sklearn.model_selection import *
from transformers import *


In [ ]:
CFG = {
    'fold_num': 5, 
    'seed': 42,
    'model': '../input/roberta-base',
    'max_len': 512,
    'epochs': 5,
    'train_bs': 24,
    'valid_bs': 32,
    'lr': 2e-5,
    'num_workers': 0,
    'weight_decay': 1e-6,
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['seed'])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
label_list = ['o', 'B-Lead', 'I-Lead', 'B-Position', 'I-Position', 'B-Claim', 'I-Claim', 'B-Counterclaim', 'I-Counterclaim', 
          'B-Rebuttal', 'I-Rebuttal', 'B-Evidence', 'I-Evidence', 'B-Concluding Statement', 'I-Concluding Statement']
label_encoding_dict = {'o': 1,
                       'B-Lead': 2,
                       'I-Lead': 3,
                       'B-Position': 4,
                       'I-Position': 5,
                       'B-Claim': 6,
                       'I-Claim': 7,
                       'B-Counterclaim': 8, 'I-Counterclaim': 9,
                       'B-Rebuttal': 10, 'I-Rebuttal': 11,
                       'B-Evidence': 12, 'I-Evidence': 13, 'B-Concluding Statement': 14,
                       'I-Concluding Statement' :15
                       
                      }


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CFG['model'], add_prefix_space=True)

### Inference Loop

In [ ]:
model =  AutoModelForTokenClassification.from_pretrained(CFG['model'], num_labels=15).to(device)
model.load_state_dict(torch.load('../input/robertafeedbackbaseline/roberta-baseline.pt'))
model.eval()

In [ ]:
test_names, test_texts = [], []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/test'))):
    test_names.append(f.replace('.txt', ''))
    test_texts.append(open('../input/feedback-prize-2021/test/' + f, 'r').read())
test_texts = pd.DataFrame({'id': test_names, 'text': test_texts})
test_texts['text'] = test_texts['text'].apply(lambda x:x.split())
test_texts

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        text = self.df.text.values[idx]
        
        return text
def collate_fn(data):
    tokenized_inputs = tokenizer(
        data,
        max_length=CFG['max_len'],
        padding='max_length',
        truncation=True,
        is_split_into_words=True,
        return_tensors='pt'
    )

    words = []
    for i in range(len(data)):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        words.append(word_ids)

    tokenized_inputs["word_ids"] = words
    return tokenized_inputs


In [ ]:
test_loader = DataLoader(MyDataset(test_texts), batch_size=CFG['valid_bs'], collate_fn=collate_fn, shuffle=False, num_workers=2)
batch = next(iter(test_loader))
batch

In [ ]:
y_pred = []
words = []

with torch.no_grad():
    tk = tqdm(test_loader, total=len(test_loader), position=0, leave=True)
    for step, batch in enumerate(tk):
        word_ids = batch['word_ids']
        words.extend(word_ids)
        batch = {k: v.to(device) for k, v in batch.items() if k != 'word_ids'}

        output = model(**batch).logits

        y_pred.extend(output.argmax(-1).cpu().numpy())
        
y_pred = np.array(y_pred)

In [ ]:
final_preds = []

for i in tqdm(range(len(test_texts))):
    idx = test_texts.id.values[i]
    pred = ['']*len(test_texts.text.values[i])

    for j in range(len(y_pred[i])):
        if words[i][j] != None:
            pred[words[i][j]] = label_list[y_pred[i][j]]

    pred = [x.replace('B-','').replace('I-','') for x in pred]

    preds = []
    j = 0
    while j < len(pred):
        cls = pred[j]
        if cls == 'o':
            j += 1
        end = j + 1
        while end < len(pred) and pred[end] == cls:
            end += 1
            
        if cls != 'o' and cls != '' and end - j > 10:
            final_preds.append((idx, cls, ' '.join(map(str, list(range(j, end))))))
        
        j = end
        
final_preds[0]

In [ ]:
test_df = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')
sub = pd.DataFrame(final_preds)
sub.columns = test_df.columns
sub

In [ ]:
sub.to_csv('submission.csv', index=False)


### Please upvote if you find the notebook useful